In [16]:
# here
!nvidia-smi

Sun Aug  3 03:07:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:00:0B.0 Off |                    0 |
| N/A   47C    P0             51W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Chai dataset - super clean and nice.

In [2]:
from datasets import load_dataset

# Load the dataset in streaming mode
ds = load_dataset("tiendung/chai", split="train", streaming=True)

# Filter to only examples with thumbs_up == True
filtered_ds = ds.filter(lambda x: x.get("thumbs_up") is True)

# Preview an example
for example in filtered_ds.take(1):
    print(example)

{'_id': '_bot_5c9283de-131d-4600-9818-11456f7458ca_mk4NsyDQzWSsWjxPYkdBOegmJit2_1689211870273_1689211909', 'conversation': [{'name': 'Barbie Roberts (Super Kind)', 'text': 'PACE!'}, {'name': 'Anonymous user', 'text': 'corn on the cob'}, {'name': 'Barbie Roberts (Super Kind)', 'text': '*laughs* what did you just say?'}], 'languages': ['en'], 'thumbs_up': True, 'feedback': 'corn', 'model_name': 'chaiml-ak-edit-v0_1689207647'}


In [9]:
from datasets import load_dataset

# Load dataset in streaming mode and filter
ds = load_dataset("tiendung/chai", split="train", streaming=True)
filtered_ds = ds.filter(lambda x: x.get("thumbs_up") is True)

def convert_chai(example):
    convo = example["conversation"]
    if not convo or len(convo) < 3:
        return []

    bot_name = convo[0]["name"]
    bot_intro = convo[0]["text"]

    messages = []

    # First user message includes bot intro
    if convo[1]["name"] != bot_name:
        user_first = convo[1]["text"]
        messages.append({
            "role": "user",
            "content": f"{bot_name}'s Persona: {bot_intro} \n<START>\nYou: {user_first}"
        })

    # First assistant message
    if len(convo) > 2 and convo[2]["name"] == bot_name:
        messages.append({
            "role": "assistant",
            "content": f"{bot_name}: {convo[2]['text']}"
        })

    # Rest of the conversation
    for i in range(3, len(convo), 2):
        # User message
        if i < len(convo) and convo[i]["name"] != bot_name:
            messages.append({
                "role": "user",
                "content": f"You: {convo[i]['text']}"
            })

        # Assistant message
        if i + 1 < len(convo) and convo[i + 1]["name"] == bot_name:
            messages.append({
                "role": "assistant",
                "content": f"{bot_name}: {convo[i + 1]['text']}"
            })

    return {"messages": messages}

# # Preview a few examples
# for example in filtered_ds.take(3):
#     formatted = convert_conversation(example)
#     for msg in formatted:
#         print(msg)
#     print("=" * 60)

all_examples = []
for example in filtered_ds:
    # text = example
    messages = convert_chai(example)
    if messages and messages['messages'] != []:  # skip empty ones
        all_examples.append(messages)

In [13]:
import json

with open("../data/chai_conversations_filtered_39k.jsonl", "w") as f:
    for item in all_examples:
        f.write(json.dumps(item) + "\n")

In [2]:
import json

with open("../data/chai_conversations_filtered_39k.jsonl", "r") as f:
    loaded_examples = [json.loads(line) for line in f]


In [3]:
loaded_examples[1]

{'messages': [{'role': 'user',
   'content': 'Jolyne Cujoh [Cell roomate]\'s Persona: *You have commited a mass crime and have been sent right to Dolphin Street Prison in Florida. You soon got to your new cell and then you heard a voice from the cell bed. You looked up and saw a hot chick, with a star shaped birthmark on her back. you truly got turned on by seeing her but decided to hide it for now. You soon found out her name was \'Jolyne Cujoh\'* (Use " " for better experience*\n\n"Oh-ho? Are you my new cell roomate?"*She looked up and down at you*"Since Gwess left the prison i have been pretty lonely. its great to have company" \n<START>\nYou: hello 😊'},
  {'role': 'assistant',
   'content': 'Jolyne Cujoh [Cell roomate]: "Hello there!" *her smile lightens the atmosphere a lot.*'}]}